In [6]:
import logic.pulsed.pulse_objects as po
from logic.pulsed.sampling_functions import SamplingFunctions as SF
import time
import matplotlib.pyplot as plt
pjl = pulsedjupyterlogic_AWG #Make sure the Logic is started in the Qudi manager!!!!

#### Current  philosophy of pulse sequence creation
1. For all the pulsed measurement that involve possible multiple phases and different pulse lengths - it is preferred and also implement right now - a long AWG sequence covering all taus is run with a single trigger from the PulseStreamer

2. For T1 sequence since it is very long and we have issues possibly due to memory - we individually trigger the AWG pi pulse before every needed pulse with the AWG sync time added in - this is also how the PODMR logic is implemented (albeit for many different freqs. so actually a different ensemble is being triggered unlike T1) - uses the sample_load_multi_short_sine_AWG function.

3. This is also valid for any other pulse sequence imaginable where individual triggering would be preferred - as long as the length of the triggerable ensembles are equal or only single ensemble is necessary.

4. Both cam be run in AWG switch mode = True (Zaswa is constantly on and AWG control the pulsing - signficantly better pulse shape and results compared to the other mode - highly recommended!).

5. Tracking mode for qAFM is also implemented such that the MW switch is always on and the AWG is controlling the on - off times

6. MW_0 and all _0 is associated with the primary (SMBV) MW source that will drive NV electron spin. MW_1 is for the secondary
SGS100A device.

In [2]:
afm_scanner_logic._mw.off()
afm_scanner_logic._mw1.off()

0

#### Define general measurement params

In [7]:
#measurement parameters
laser_power_voltage = podmrlogic.laser_power_voltage

#Driving information for the first LO (SMBV)
target_freq_0 = 4.6319e9
LO_freq_0 = target_freq_0 + 100e6
power_0 = -1
pi_pulse = 103e-9 

#For same NV rabi SGS is run with power_1 = power_0 - 11dBm
#Driving information for the second LO (SGS)
target_freq_1 = 2.87e9
LO_freq_1 = target_freq_1 + 100e6
power_1 = -100
pi_pulse_1 = 72e-9

switch_MW = False #If True, MW1 is used as the main MW source

#additional information for save tag
tip_name = 'V-J13-23'
sample = 'NbSe2_S2'
temperature = '2K'
b_field = '70mT_OOP'
contact = 'OOC'
extra = ''

#### Load and ready Rabi sequence

In [8]:
tau_start = 0
tau_stop = 400e-9
tau_num = 50

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Rabi(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Rabi', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready PODMR sequence

In [17]:
mw_start = 2.41e9
mw_stop = 2.45e9
mw_step = 1e6

if LO_freq_0<mw_stop+mw_step:
    print('!!!Warning! LO Freq too low. Sweep is too large in range!!!\n\n\n')
pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.PODMR(mw_start, mw_stop, mw_step)
pjl.start_measurement(measurement_type='PODMR', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready Tracking mode sequence

In [4]:
res_freq = target_freq_0
delta_freq = 5e6

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Tracking(res_freq, delta_freq)
pjl.start_measurement(measurement_type='Tracking', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready Charge dynamic sequence

In [4]:
waiting_time = 1e-6

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Charge_dynamic_alt(waiting_time)
pjl.start_measurement(measurement_type='Charge-dynamic', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready optical T1 sequence

In [2]:
tau_start = 1e-6
tau_stop = 1e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.T1_optical_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-opt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready T1 sequence (alternating)

In [5]:
tau_start = 1e-6
tau_stop = 5e-6
tau_num = 2

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.T1_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready T1 sequence init dark (alternating)

In [24]:
tau_start = 1e-6
tau_stop = 30e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.T1_dark_init_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-dark-init-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready T1 SQ sequence (alternating)

In [33]:
tau_start = 1e-6
tau_stop = 40e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.T1_SQ_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-SQ-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54057 GHz
Using target_freq:  2.44057 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.74668135e-06 3.05089575e-06 5.32894271e-06
 9.30796486e-06 1.62580486e-05 2.83976304e-05 4.96016115e-05
 8.66382098e-05 1.51329346e-04 2.64324146e-04 4.61690057e-04
 8.06425413e-04 1.40856823e-03 2.46031986e-03 4.29739482e-03
 7.50617940e-03 1.31109036e-02 2.29005708e-02 4.00000000e-02]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-Pi']
Save_tag:  T1-SQ-alt_A-T17-12_1.9K_200mT_Bnv_OOC_Set_A


#### Load and ready T1 DQ sequence (alternating)

In [13]:
tau_start = 1e-6
tau_stop = 1e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.T1_DQ_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-DQ-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.07 V
Using LO_freq:  2.881845 GHz
Using target_freq:  2.78469 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.43844989e-06 2.06913808e-06 2.97635144e-06
 4.28133240e-06 6.15848211e-06 8.85866790e-06 1.27427499e-05
 1.83298071e-05 2.63665090e-05 3.79269019e-05 5.45559478e-05
 7.84759970e-05 1.12883789e-04 1.62377674e-04 2.33572147e-04
 3.35981829e-04 4.83293024e-04 6.95192796e-04 1.00000000e-03]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
Save_tag:  T1-DQ-alt_A-T17-12_1.9K_0mT_Bnv_OOC


#### Load and ready Ramsey sequence with phase shift (alternating)

In [39]:
tau_start = 0
tau_stop = 10e-6
tau_num = 200

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Ramsey_alt_phased(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Ramsey-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready Hahn Echo sequence with phase shift (alternating)

In [23]:
tau_start = 200e-9
tau_stop = 20e-6
tau_num = 200

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Hecho_alt_phased(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Hecho-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready Hahn Echo correlation sequence with phase shift (alternating with one long segment)

In [30]:
hecho_waiting = 10e-6
tau_start = 1e-6
tau_stop = 100e-6
tau_num = 500

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Hecho_correlation_alt_phased(hecho_waiting, tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Hecho-correlation-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=f'_Hahnechowaitingtime={hecho_waiting}s_'+extra)

#### Load and ready Hahn Echo correlation sequence with phase shift and exponential spacing (alternating with one long segment)

In [4]:
hecho_waiting = 1.7e-6
tau_start = 1e-6
tau_stop = 100e-6
tau_num = int(500/8)

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.Hecho_correlation_alt_phased_exp(hecho_waiting, tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Hecho-correlation-alt-phased-exp', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=f'_Hahnechowaitingtime={hecho_waiting}s_'+extra)

#### Load and ready Hahn Echo correlation sequence with phase shift (alternating with several short segments, organized in the sequence step list)

In [3]:
hecho_waiting = 2e-6
tau_start = 1e-6
tau_stop = 100e-6
tau_num = 500

# pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
# pjl.Hecho_correlation_alt_phased_short(hecho_waiting, tau_start, tau_stop, tau_num)
# pjl.start_measurement(measurement_type='Hecho-correlation-alt-phased-short', tip_name=tip_name, sample=sample, 
#                       temperature=temperature, b_field=b_field, contact=contact, extra=extra)

([['Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-hecho_init'], ['Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-hecho_read_out'], ['Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-hecho_read_out_alt'], ['Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-waiting']], [{'step_index': 0, 'step_segment': 'Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-hecho_init', 'step_loops': 1, 'next_step_index': 1, 'step_end_cond': 'always'}, {'step_index': 1, 'step_segment': 'Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-waiting', 'step_loops': 3, 'next_step_index': 2, 'step_end_cond': 'always'}, {'step_index': 2, 'step_segment': 'Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-hecho_read_out', 'step_loops': 1, 'next_step_index': 3, 'step_end_cond': 'always'}, {'step_index': 3, 'step_segment': 'Jupyter-ensemble-hecho-correlation-alt-phased-short-juptr-hecho_init', 'step_loops': 1, 'next_step_index': 4, 'step_end_cond': 'always'}

#### Load and ready DEER sequence with alternating (no dark spin flip)

In [8]:
mw_start = 5.175e9
mw_stop = 5.40e9
mw_step = 2e6
tau_NV = 16e-6
pi_pulse_dark_spin = tau_NV/2*0.9

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.DEER_alt(tau_NV=tau_NV, pi_pulse_dark_spin=pi_pulse_dark_spin, mw_start=mw_start, mw_stop=mw_stop, mw_step=mw_step)
pjl.start_measurement(measurement_type='DEER-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready DEER sequence with phase shift (alternating NV)

In [6]:
mw_start = 5.175e9
mw_stop = 5.40e9
mw_step = 2e6
tau_NV = 16e-6
pi_pulse_dark_spin = tau_NV/2*0.9

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.DEER_alt_phased(tau_NV=tau_NV, pi_pulse_dark_spin=pi_pulse_dark_spin, mw_start=mw_start, mw_stop=mw_stop, mw_step=mw_step)
pjl.start_measurement(measurement_type='DEER-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready DEER Rabi sequence with phase shift (alternating)

In [5]:
freq_dark_spin = 5.2907e9

tau_NV = 16e-6
pi_pulse_dark_spin = tau_NV/2*0.9

ds_tau_start = 10e-9
ds_tau_stop = pi_pulse_dark_spin
ds_tau_num = 100

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.DEER_rabi_alt_phased(tau_NV=tau_NV, freq_dark_spin=freq_dark_spin, dark_spin_tau_start=ds_tau_start, dark_spin_tau_stop=ds_tau_stop, dark_spin_tau_num=ds_tau_num)
pjl.start_measurement(measurement_type='DEER-rabi-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready DEER CPMG sequence with phase shift (alternating)

In [10]:
mw_start = 5.3e9
mw_stop = 5.62e9
mw_step = 1e6
tau_NV = 10e-6
N = 16
pi_pulse_dark_spin = (tau_NV- 2*N*pi_pulse)/(2*N)

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
pjl.DEER_CPMG_alt_phased(tau_NV=tau_NV, pi_pulse_dark_spin=pi_pulse_dark_spin, mw_start=mw_start, mw_stop=mw_stop, mw_step=mw_step, N=N)
pjl.start_measurement(measurement_type='DEER-CPMG-alt-phased', tip_name=tip_name, sample=sample, 
                        temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready CPMG sequence (alternating)

In [6]:
tau_start = 1.5e-6
tau_stop = 30e-6
tau_num = 300
N = 8

if tau_start < 2*N*pi_pulse:
    min_tau = pi_pulse*2*N*1e9 #minimum tau in ns
    print(f'!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.CPMG_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'CPMG({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [5]:
2*8*75e-9

1.2e-06

#### Load and ready XY4 sequence (alternating)

In [ ]:
tau_start = 18e-6
tau_stop = 36e-6
tau_num = 200
N = 8

if tau_start/(4*N) < pi_pulse:
    min_tau = pi_pulse*4*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.XY4_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY4({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [ ]:
pi_pulse_spacing_start = 100e-9
pi_pulse_spacing_stop = 600e-9
pi_pulse_spacing_num = 100
N = 8

tau_start = pi_pulse_spacing_start*N*4
tau_stop = pi_pulse_spacing_stop*N*4
tau_num = pi_pulse_spacing_num


if tau_start/(4*N) < pi_pulse:
    min_tau = pi_pulse*4*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.XY4_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY4({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready XY8 sequence (alternating)

In [1]:
tau_start = 15e-6
tau_stop = 30e-6
tau_num = 300
N = 1

if tau_start/(8*N) < pi_pulse:
    min_tau = pi_pulse*8*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.XY8_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY8({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [ ]:
pi_pulse_spacing_start = 100e-9
pi_pulse_spacing_stop = 600e-9
pi_pulse_spacing_num = 100
N = 8

tau_start = pi_pulse_spacing_start*N*8
tau_stop = pi_pulse_spacing_stop*N*8
tau_num = pi_pulse_spacing_num


if tau_start/(8*N) < pi_pulse:
    min_tau = pi_pulse*8*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.XY8_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY8({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready XY16 sequence (alternating)

In [ ]:
tau_start = 18e-6
tau_stop = 36e-6
tau_num = 200
N = 8

if tau_start/(16*N) < pi_pulse:
    min_tau = pi_pulse*16*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.XY16_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY16({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [143]:
pi_pulse_spacing_start = 100e-9
pi_pulse_spacing_stop = 600e-9
pi_pulse_spacing_num = 100
N = 8

tau_start = pi_pulse_spacing_start*N*16
tau_stop = pi_pulse_spacing_stop*N*16
tau_num = pi_pulse_spacing_num


if tau_start/(16*N) < pi_pulse:
    min_tau = pi_pulse*16*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1, switch_MW = switch_MW)
    pjl.XY16_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY16({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [3]:
res_freq = target_freq_0
delta_freq = 5e6
repetitions = 150000

pjl.initialize_ensemble(laser_power_voltage = laser_power_voltage, pi_pulse=pi_pulse, pi_pulse_1=pi_pulse_1, LO_freq_0=LO_freq_0, target_freq_0=res_freq, power_0=power_0, switch_MW = switch_MW)
pjl.sample_load_ready_AWG_for_SPM_tracking(res_freq, delta_freq, repetitions)
pjl.sample_load_ready_pulsestreamer(name='read_out_jptr')
afm_scanner_logic._pulsed_master_AWG.pulsedmeasurementlogic().pulsegenerator().pulser_on()

1